In [2]:
import numpy as np
import heapq
import torch
import csv
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.spatial import distance
from PIL import Image
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader

In [3]:
class DogDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

## Импортируем датасет

In [4]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

with open('test_dataset', 'r') as csv_file:
    reader = csv.DictReader(csv_file, delimiter=',')
    names, photo_paths = [], []
    for line in reader:
        names.append(line['name'])
        photo_paths.append(line['photo_path'])

test_dataset = DogDataset(photo_paths, names, transform=transform)
photo_paths_save = photo_paths

with open('test_dataset_single', 'r') as csv_file:
    reader = csv.DictReader(csv_file, delimiter=',')
    names, photo_paths = [], []
    for line in reader:
        names.append(line['name'])
        photo_paths.append(line['photo_path'])

test_dataset_single = DogDataset(photo_paths, names, transform=transform)

## Создаем DataLoaders

In [5]:
test_dataloader = DataLoader(test_dataset_single, batch_size=1)

## Инициализируем модель

In [6]:
checkpoint = torch.load('checkpoint-000025.pth.tar', map_location=torch.device('cpu'))
state_dict = checkpoint['state_dict']

model = models.resnet50(weights='DEFAULT')
model = torch.nn.Sequential(*list(model.children())[:-1])
model.load_state_dict(state_dict)
model.eval()
plt.show()

/tmp/ipykernel_12791/3313998203.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('checkpoint-000025.pth.tar', map_location=torch.device('cpu'))


## Подготовка к тестированию

In [7]:
class AverageMeters(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.cnt = 0
        self.sum = 0
        self.val = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.cnt += n
        self.avg = self.sum / self.cnt

In [8]:
def extract_features(image_path):
    image = Image.open(image_path)
    img_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        features = model(img_tensor)
    return features.flatten()

In [9]:
embeddings = []
for photo_path in photo_paths_save:
    embeddings.append(extract_features(photo_path))

In [18]:
def accuracy(predict, target, embeddings):
    with torch.no_grad():
        similar_idx = [distance.cosine(predict, feat) for feat in embeddings]
        idx_closest = sorted(range(len(similar_idx)), key=lambda k: similar_idx[k])[1]
        correct = 1 if test_dataset[idx_closest][1] in target else 0
        accuracy = correct * 100
    return accuracy

#### Выбор девайса для работы

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
plt.show()

cpu


#### Цикл тестирования

In [20]:
def test_loop(dataloader, model):
    losses = AverageMeters()
    accs = AverageMeters()

    for _, (data, target) in enumerate(dataloader):
        data = data.to(device)

        predict = model(data).flatten()

        acc = accuracy(predict, target, embeddings)

        accs.update(acc)
    return accs.avg

In [21]:
test_loop(test_dataloader, model)

3.8043478260869565